In [5]:
from fontTools.misc.timeTools import epoch_diff
from keras import datasets, layers,utils,Sequential
import numpy as np
from keras.src.layers import Dense
from keras_tuner.tuners import RandomSearch,Hyperband,BayesianOptimization
from tensorflow.python.data.experimental.ops.distribute import batch_sizes_for_worker

In [6]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

In [7]:
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000, 784)
x_train=x_train/255
x_test=x_test/255
y_train=utils.to_categorical(y_train,10)
y_test=utils.to_categorical(y_test,10)

In [8]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation_choice', ['relu', 'tanh', 'sigmoid','elu','selu'])
    model.add(Dense(units=hp.Int('units_input', min_value=512, max_value=1024, step=32), activation=activation_choice,input_dim=784))
    model.add(Dense(units=hp.Int('units_hidden',min_value=128, max_value=600, step=32), activation=activation_choice))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', 
                  optimizer=hp.Choice('optimizer',values=['adam','rmsprop','SGD']), metrics=['accuracy'])
    return model

In [9]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=4,directory='test_directory')
tuner.search_space_summary()

Reloading Tuner from test_directory\untitled_project\tuner0.json
Search space summary
Default search space size: 4
activation_choice (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 600, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


In [10]:
tuner.search(x_train,y_train,batch_size=256,epochs=20,validation_split=0.2)

In [11]:
tuner.results_summary()

Results summary
Results in test_directory\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 75 summary
Hyperparameters:
activation_choice: relu
units_input: 992
units_hidden: 160
optimizer: adam
Score: 0.9005833268165588

Trial 59 summary
Hyperparameters:
activation_choice: relu
units_input: 704
units_hidden: 416
optimizer: adam
Score: 0.8982499837875366

Trial 31 summary
Hyperparameters:
activation_choice: relu
units_input: 960
units_hidden: 224
optimizer: adam
Score: 0.8980833292007446

Trial 37 summary
Hyperparameters:
activation_choice: elu
units_input: 992
units_hidden: 256
optimizer: adam
Score: 0.8977500200271606

Trial 20 summary
Hyperparameters:
activation_choice: tanh
units_input: 800
units_hidden: 512
optimizer: adam
Score: 0.8974166512489319

Trial 72 summary
Hyperparameters:
activation_choice: relu
units_input: 736
units_hidden: 576
optimizer: adam
Score: 0.8969166874885559

Trial 27 summary
Hyperparameters:
activation_choice: r

In [12]:
models=tuner.get_best_models(num_models=3)
for model in models:
    model.summary()
    model.evaluate(x_test,y_test)
    print()

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\ProgramData\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 992)            │       778,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 160)            │       158,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 939,210 (3.58 MB)

 Trainable params: 939,210 (3.58 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 960)            │       753,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 224)            │       215,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 971,114 (3.70 MB)

 Trainable params: 971,114 (3.70 MB)

 Non-trainable params: 0 (0.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8914 - loss: 0.3430

